# <center>Agents in Autogen</center>

In [21]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from dotenv import load_dotenv
import os
import asyncio
from openai import OpenAI
from IPython.display import Markdown, display

load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')

model_client = OpenAIChatCompletionClient(
    model="gemini-2.0-flash-exp",
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=google_api_key,
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "family": "unknown",
        "structured_output": False
    }
)

### Tools

In [22]:
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "The Persian cat is an Iranian breed of domestic cat known for its long fur and gentle temperament. "

### Agents

In [23]:
agent = AssistantAgent(
    name='assistant',
    model_client=model_client,
    tools=[web_search],
    system_message='Use Tools to solve tasks',
    description='An agent which uses tool to help solve task'
)

In [24]:
result = await agent.run(task='Find information about Persian cat')
display(Markdown(result.messages[-1].content))

The Persian cat is an Iranian breed of domestic cat known for its long fur and gentle temperament. 

In [25]:
print(result.messages)

[TextMessage(id='aee17ef6-50a9-4650-96b1-666846f7fb77', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 1, 11, 53, 40, 451459, tzinfo=datetime.timezone.utc), content='Find information about Persian cat', type='TextMessage'), ToolCallRequestEvent(id='9e6660fb-3608-4154-be4c-12a12bde186a', source='assistant', models_usage=RequestUsage(prompt_tokens=23, completion_tokens=6), metadata={}, created_at=datetime.datetime(2025, 8, 1, 11, 53, 41, 284592, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='', arguments='{"query":"Persian cat"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='0f2c5878-5086-41cd-86ef-6e06237e6bf0', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 1, 11, 53, 41, 285598, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The Persian cat is an Iranian breed of domestic cat known for its long fur and gentle temperament. ', name='web_se

TaskResult(messages= [ 
    
1. TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 19, 16, 0, 26, 684759, tzinfo=datetime.timezone.utc), content='Find information about Labrador Retriever', type='TextMessage')

2. ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=61, completion_tokens=17), metadata={}, created_at=datetime.datetime(2025, 6, 19, 16, 0, 27, 992815, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_ipWxcrV4tXyUsapUhqpz6NyO', arguments='{"query":"Labrador Retriever"}', name='web_search')], type='ToolCallRequestEvent'), 

3. ToolCallExecutionEvent(source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 19, 16, 0, 27, 993741, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The Labrador Retriever or simply Labrador is a British breed of retriever gun dog. ', name='web_search', call_id='call_ipWxcrV4tXyUsapUhqpz6NyO', is_error=False)], type='ToolCallExecutionEvent'), 

4. ToolCallSummaryMessage(source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 19, 16, 0, 27, 993775, tzinfo=datetime.timezone.utc), content='The Labrador Retriever or simply Labrador is a British breed of retriever gun dog. ', type='ToolCallSummaryMessage', tool_calls=[FunctionCall(id='call_ipWxcrV4tXyUsapUhqpz6NyO', arguments='{"query":"Labrador Retriever"}', name='web_search')], 

results=[FunctionExecutionResult(content='The Labrador Retriever or simply Labrador is a British breed of retriever gun dog. ', name='web_search', call_id='call_ipWxcrV4tXyUsapUhqpz6NyO', is_error=False)])], stop_reason=None)

### On_message() method

In [26]:
from autogen_core import CancellationToken

async def assistant_run()-> None:
    response = await agent.on_messages(
        messages= [TextMessage(content='Find information about Persian cat',source='User')],
        cancellation_token=CancellationToken()
    )

    print(response.inner_messages)
    print('\n\n\n\n')
    print(response.chat_message)

await assistant_run()

[]





id='aeaacafb-f218-454d-9d91-a88c77798994' source='assistant' models_usage=RequestUsage(prompt_tokens=58, completion_tokens=20) metadata={} created_at=datetime.datetime(2025, 8, 1, 11, 53, 42, 520967, tzinfo=datetime.timezone.utc) content='The Persian cat is an Iranian breed of domestic cat known for its long fur and gentle temperament.\n' type='TextMessage'


### Streaming Messages
on_messages_stream()

In [27]:
from autogen_agentchat.ui import Console


async def assistant_run_stream() -> None:

    await Console(
        agent.on_messages_stream(
        messages= [TextMessage(content='Find information about Labrador Retriever via the tool',source='User')],
        cancellation_token=CancellationToken()
    ),
    output_stats=True # Enable stats Printing
    )


await assistant_run_stream()

---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='', arguments='{"query":"Labrador Retriever"}', name='web_search')]
[Prompt tokens: 86, Completion tokens: 7]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='The Persian cat is an Iranian breed of domestic cat known for its long fur and gentle temperament. ', name='web_search', call_id='', is_error=False)]
---------- assistant ----------
The Persian cat is an Iranian breed of domestic cat known for its long fur and gentle temperament. 
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 86
Total completion tokens: 7
Duration: 0.68 seconds


In [28]:
from autogen_agentchat.ui import Console


async def assistant_run_stream() -> None:

    await Console(
        agent.on_messages_stream(
        messages= [TextMessage(content='What was the last question I asked ?',source='User')],
        cancellation_token=CancellationToken()
    ),
    output_stats=True # Enable stats Printing
    )


await assistant_run_stream()

---------- assistant ----------
You asked me to find information about Labrador Retriever via the tool.

[Prompt tokens: 125, Completion tokens: 14]
---------- Summary ----------
Number of inner messages: 0
Total prompt tokens: 125
Total completion tokens: 14
Duration: 1.04 seconds
